In [3]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Un

In [6]:
!pip install pyngrok flask-ngrok

In [8]:
!ngrok authtoken 1vPKimF2WGxymKk9UAn8hokTwyk_3GXKRhPzJsULvRcXrV9Tz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from flask import Flask, render_template, request, redirect, url_for
from PIL import Image, ImageOps
from keras.utils import image_utils
import os
import numpy as np
import io
import base64
from pyngrok import ngrok

# Assuming loaded_models and class_labels are already defined
IMAGE_SIZE = (224, 224)  # Adjust according to your model's input size
# loaded_models = your_model_loading_function()  # Replace with your actual model loading

# Snake information dictionary
snake_info_dict = {
    "Branded Krait": {
        "description": "The Branded Krait is a slender, highly venomous snake known for its distinct black and white bands.",
        "habitat": "Found in forests, grasslands, and agricultural areas across Southeast Asia, particularly in India and parts of Indonesia.",
        "diet": "Primarily feeds on other snakes, as well as small mammals and lizards.",
        "venomous": "Yes, it is highly venomous and can be fatal to humans if untreated."
    },
    "Golden Tree Snake": {
        "description": "The Golden Tree Snake is a non-venomous arboreal snake known for its striking yellow and green coloration.",
        "habitat": "Commonly found in tropical rainforests and other lush environments in Southeast Asia.",
        "diet": "Feeds mainly on small lizards and frogs.",
        "venomous": "No, it is non-venomous and poses little threat to humans."
    },
    "King Cobra": {
        "description": "The King Cobra is the longest venomous snake in the world, capable of reaching lengths of up to 18 feet.",
        "habitat": "Typically inhabits forests, grasslands, and bamboo thickets in Southeast Asia and parts of India.",
        "diet": "Primarily preys on other snakes, including venomous species.",
        "venomous": "Yes, it has a highly toxic venom that can cause death if untreated."
    },
    "Kukri Snake": {
        "description": "The Kukri Snake is a non-venomous snake recognized for its distinct, curved teeth resembling a kukri knife.",
        "habitat": "Found in a variety of habitats, including forests and grasslands across the Indian subcontinent and Southeast Asia.",
        "diet": "Feeds on amphibians and small reptiles.",
        "venomous": "No, it is non-venomous and harmless to humans."
    },
    "Long-nosed Whip Snake": {
        "description": "The Long-nosed Whip Snake is a slender, agile snake known for its long, pointed snout and excellent climbing ability.",
        "habitat": "Typically found in forests and shrubs, primarily in Central and South America.",
        "diet": "Feeds on small lizards and insects.",
        "venomous": "No, it is non-venomous."
    },
    "Reticulated Python": {
        "description": "The Reticulated Python is one of the largest snakes in the world, known for its intricate patterns.",
        "habitat": "Found in rainforests, woodlands, and grasslands across Southeast Asia.",
        "diet": "Feeds on a variety of prey, including birds, mammals, and occasionally pigs and deer.",
        "venomous": "No, it is non-venomous."
    },
    "Siamese Russell's Viper": {
        "description": "The Siamese Russell's Viper is a venomous snake characterized by its heavy body and distinctive markings.",
        "habitat": "Commonly found in grasslands, forests, and agricultural areas in Southeast Asia.",
        "diet": "Feeds on small mammals, birds, and amphibians.",
        "venomous": "Yes, it is highly venomous."
    }
}


app = Flask(__name__)

IMAGE_SIZE = (224, 224)

def classify_snake(image):
    try:
        # Resize and preprocess the image
        image = ImageOps.fit(image, IMAGE_SIZE, Image.LANCZOS)
        x = image_utils.img_to_array(image) / 255.0
        x = np.expand_dims(x, axis=0)

        # Predict using the model
        preds = loaded_models.predict(x)
        pred_class = np.argmax(preds)
        confidence = np.max(preds) * 100

        # Return predicted class and confidence score (formatted to 2 decimal places)
        return class_labels[pred_class], f"{confidence:.2f}"
    except Exception as e:
        print(f"Error during classification: {e}")
        return None, None

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if request.method == 'POST':
        # Handle file upload
        image = request.files['file']
        image = Image.open(image).convert('RGB')

        # Perform classification
        snake_name, confidence = classify_snake(image)
        snake_info = snake_info_dict.get(snake_name, {})

        # Ensure the directory exists
        image_folder = os.path.join('static', 'images')
        if not os.path.exists(image_folder):
            os.makedirs(image_folder)

        # Save the uploaded image or get its URL
        image_path = os.path.join(image_folder, 'uploaded_image.jpg')
        image.save(image_path)
        image_url = url_for('static', filename='images/uploaded_image.jpg')

        # Render the results page
        return render_template('result.html', image_url=image_url,
                               snake_name=snake_name,
                               confidence=confidence,
                               snake_info=snake_info)
    return redirect(url_for('index'))


if __name__ == "__main__":
    # Start ngrok to expose the app
    public_url = ngrok.connect(5000)
    print(f" * Running on {public_url}")

    # Run the Flask app
    app.run(host="0.0.0.0", port=5000)


 * Running on NgrokTunnel: "https://0e9d-34-125-242-206.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Oct/2024 15:47:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Oct/2024 15:47:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Oct/2024 15:47:37] "POST /upload HTTP/1.1" 200 -


Error during classification: name 'loaded_models' is not defined


INFO:werkzeug:127.0.0.1 - - [13/Oct/2024 15:47:37] "GET /static/images/uploaded_image.jpg HTTP/1.1" 200 -
